In [12]:
#importing files and libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
API_key = 'J4SJ6FAAQG63WZLC'

In [13]:
#getting technical Indicators from API
ta = TechIndicators(key = API_key, output_format = 'pandas')

In [14]:
#getting SMA of stock 
goog, meta = ta.get_sma('GOOG', interval = 'daily', time_period= 200, series_type= 'close') 

In [15]:
# Preparing Dataset

#fetching TimeSeries data for company
ts = TimeSeries(key = API_key, output_format = 'pandas')
data = ts.get_daily_adjusted('TSLA', outputsize= 'full')    
f_data = data[0]

#renaming columns
f_data.rename(columns={'1. open':'Open', '2. high': 'High', '3. low':'Low', '4. close':'Close', '5. adjusted close': 'Adjusted Close', '6. volume':'Volume', '7. dividend amount':'Dividend Amount', '8. split coefficient':'Split Coefficient'}, inplace = True)
f_data.head(3)

#dropping unwanted columns
f_data.drop(['Split Coefficient'], axis= 1, inplace=True)
f_data.drop(['Dividend Amount'], axis = 1, inplace = True)
f_data.drop(['Adjusted Close'], axis= 1, inplace= True)

#making a column for Rise/Drop
f_data['Rise/Drop'] = f_data['Open'] - f_data['Close']
f_data.sort_index(inplace=True)

#matching the TS data to SMA data
f_data = f_data.loc['2015-01-09':]  

# Plotting graphs 

In [16]:
#SMA
fig1 = px.area(goog, y="SMA", title= "Simple Moving Average")
fig1.update_layout(title=dict(x=0.5))
fig1.update_traces(line=dict(color='green', width=3))
fig1.update_layout(legend=dict(x=0, y=1, traceorder="normal"))
fig1.add_annotation(x=goog['SMA'].idxmax(), y=goog['SMA'].max(), text=f'{int(goog.max())} is the highest value', showarrow=True, arrowhead=1)
fig1.show()

In [17]:
#Opening / Closing price 
f_data_new = f_data.reset_index()
# Initialize figure
fig = go.Figure()

# Add Traces

fig.add_trace(
    go.Scatter(x=list(f_data_new.date),
               y=list(f_data_new.High),
               name="High",
               line=dict(color="#33CFA5")))

fig.add_trace(
    go.Scatter(x=list(f_data_new.date),
               y=[f_data_new.High.mean()] * len(f_data_new.index),
               name="High Average",
               visible=False,
               line=dict(color="#33CFA5", dash="dash")))

fig.add_trace(
    go.Scatter(x=list(f_data_new.date),
               y=list(f_data_new.Low),
               name="Low",
               line=dict(color="#F06A6A")))

fig.add_trace(
    go.Scatter(x=list(f_data_new.date),
               y=[f_data_new.Low.mean()] * len(f_data_new.index),
               name="Low Average",
               visible=False,
               line=dict(color="#F06A6A", dash="dash")))

# Add Annotations and Buttons
high_annotations = [dict(x="2016-03-01",
                         y=f_data_new.High.mean(),
                         xref="x", yref="y",
                         text="High Average:<br> %.3f" % f_data_new.High.mean(),
                         ax=0, ay=-40),
                    dict(x=f_data_new.date[f_data_new.High.idxmax()],
                         y=f_data_new.High.max(),
                         xref="x", yref="y",
                         text="High Max:<br> %.3f" % f_data_new.High.max(),
                         ax=-40, ay=-40)]
low_annotations = [dict(x="2015-05-01",
                        y=f_data_new.Low.mean(),
                        xref="x", yref="y",
                        text="Low Average:<br> %.3f" % f_data_new.Low.mean(),
                        ax=0, ay=40),
                   dict(x=f_data_new.date[f_data_new.High.idxmin()],
                        y=f_data_new.Low.min(),
                        xref="x", yref="y",
                        text="Low Min:<br> %.3f" % f_data_new.Low.min(),
                        ax=0, ay=40)]

fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=list([
                dict(label="None",
                     method="update",
                     args=[{"visible": [True, False, True, False]},
                           {"title": "Google",
                            "annotations": []}]),
                dict(label="High",
                     method="update",
                     args=[{"visible": [True, True, False, False]},
                           {"title": "Google High",
                            "annotations": high_annotations}]),
                dict(label="Low",
                     method="update",
                     args=[{"visible": [False, False, True, True]},
                           {"title": "Google Low",
                            "annotations": low_annotations}]),
                dict(label="Both",
                     method="update",
                     args=[{"visible": [True, True, True, True]},
                           {"title": "Google",
                            "annotations": high_annotations + low_annotations}]),
            ]),
        )
    ])

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Set title
fig.update_layout(title_text="Google")

fig.show()

In [18]:
# Rise / Drop trend
data = f_data.groupby(pd.Grouper(freq='M')).mean('Rise/Drop')    #change to make a function and add filter 
fig_data = px.area(data, y = 'Rise/Drop', title = "Rise - Drop Trend",)
fig_data.update_layout(title=dict(x=0.5))

# Add range slider
fig_data.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)


#adding horizontal line
fig_data.add_shape(type='line', x0=0, x1=1, xref='paper', y0=0, y1=0, yref='y')
fig_data.show()

In [19]:
#candle-Stick Chart
cs_df = f_data
cs_df = cs_df.reset_index()

figure = make_subplots(specs=[[{"secondary_y": True}]] )
figure.add_trace(go.Candlestick(
    x=cs_df['date'],
    open=cs_df['Open'],
    high=cs_df['High'],
    low=cs_df['Low'],
    close=cs_df['Close']), 
    secondary_y= True)

# Add range slider
figure.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

figure.add_trace(go.Bar(x = cs_df["date"], y = cs_df["Volume"]), secondary_y= False)
figure.show()

In [2]:
import requests
API_key = 'J4SJ6FAAQG63WZLC'

url = "https://www.alphavantage.co/query?function=LISTING_STATUS&apikey=" + API_key

<Response [200]>


In [15]:
import pandas as pd
stock_names = pd.read_csv(url)

In [18]:
stock_names[['name','symbol']].T.to_dict()

{0: {'name': 'Agilent Technologies Inc', 'symbol': 'A'},
 1: {'name': 'Alcoa Corp', 'symbol': 'AA'},
 2: {'name': 'AXS FIRST PRIORITY CLO BOND ETF ', 'symbol': 'AAA'},
 3: {'name': 'Goldman Sachs Physical Gold ETF', 'symbol': 'AAAU'},
 4: {'name': 'Ares Acquisition Corporation - Class A', 'symbol': 'AAC'},
 5: {'name': 'Ares Acquisition Corporation - Units (1 Ord Share Class A & 1/5 War)',
  'symbol': 'AAC-U'},
 6: {'name': 'Ares Acquisition Corporation - Warrants (01/01/9999)',
  'symbol': 'AAC-WS'},
 7: {'name': 'ATA Creativity Global', 'symbol': 'AACG'},
 8: {'name': 'Armada Acquisition Corp I', 'symbol': 'AACI'},
 9: {'name': 'Armada Acquisition Corp I - Units (1 Ord & 1/2 War)',
  'symbol': 'AACIU'},
 10: {'name': 'Armada Acquisition Corp I - Warrants (13/08/2026)',
  'symbol': 'AACIW'},
 11: {'name': 'Aadi Bioscience Inc', 'symbol': 'AADI'},
 12: {'name': 'AdvisorShares Dorsey Wright ADR ETF', 'symbol': 'AADR'},
 13: {'name': 'Arlington Asset Investment Corp - Class A', 'symbol':